In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max, min, to_date
from pyspark.ml.regression import LinearRegression
# from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import os
import shapefile
from pmdarima.arima import auto_arima

In [2]:
#Create Spark Session
spark = SparkSession.builder.appName("FlubyRegion").getOrCreate()

23/11/01 14:50:12 WARN Utils: Your hostname, cyber-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/11/01 14:50:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/01 14:50:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
root_dir = "flu_region"
data_dir = "data"

In [56]:
#Divide Flu-by-Region into Flu-by-County
filecheck1 = f"{root_dir}/flu_by_county.csv"
if not os.path.isfile(filecheck1):
    fbc_dict = {
    "Bay Area": ["01675839" , "01675903", "00277285", "00277292", "00277302", "00277305" , "00277307",  "00277308" , "00277312", "01657246"],
    "Central": ["01675885", "00277274", "01804637", "00277280", "00277284", "00277286", "00277288", "00277290", "00277291", "00277299", "00277303", "00277314", "00277318", "00277319"],
    "Lower Southern" : ["00277277", "00277294", "00277297", "00277300", "00277301"],
    "Northern" : ["01675840", "01675841", "01675842", "01675902", "01682074", "00277273", "00277275", "01681908", "00277281", "01693324", "00277287", "00277289", "01682927", "00277295", "00277296",  "00277298", "01682610", "00277310", "00277311", "00277315", "01692767", "00277317", "00277321", "00277322"],
    "Upper Southern" : ["02054176", "00277283", "00277304", "00277306", "00277320"]
    }

    df = pd.read_csv(f"{data_dir}/flu-ili-byregion-fluseason.csv")
    # Create an empty list to store new dataframes
    new_dfs = []

    # Loop through each row in the original DataFrame
    for _, row in df.iterrows():
        x_value = row["region"]
        values_to_replace = fbc_dict.get(x_value, [x_value])

        # Create new dataframes for each value in values_to_replace
        for new_x_value in values_to_replace:
            new_df = row.copy().to_frame().T  # Create a new DataFrame with one row
            new_df["region"] = new_x_value
            new_dfs.append(new_df)

    # Concatenate the list of dataframes into a single dataframe
    new_df = pd.concat(new_dfs, ignore_index=True)

    # Write the modified DataFrame to a CSV file
    new_df.to_csv(f"{root_dir}/flu_by_county.csv", index=False)

df = spark.read.csv(f"{root_dir}/flu_by_county.csv", header=True, inferSchema=True)
df.show()

+---------+---------+----------+----------+---------+-------------------+-----------+--------------------------+
|   season|date_code|weekending|    region|Total_ILI|Total_Patients_Seen|Percent_ILI|Number_Providers_Reporting|
+---------+---------+----------+----------+---------+-------------------+-----------+--------------------------+
|2001-2002|   200140| 10/6/2001|  01675839|        3|                135|       2.22|                         2|
|2001-2002|   200140| 10/6/2001|  01675903|        3|                135|       2.22|                         2|
|2001-2002|   200140| 10/6/2001|  00277285|        3|                135|       2.22|                         2|
|2001-2002|   200140| 10/6/2001|  00277292|        3|                135|       2.22|                         2|
|2001-2002|   200140| 10/6/2001|  00277302|        3|                135|       2.22|                         2|
|2001-2002|   200140| 10/6/2001|  00277305|        3|                135|       2.22|           

In [5]:
#Divide data into separate files by date
filecheck2 = f"{root_dir}/flu_by_dates"
if not os.listdir(filecheck2):
    # Create a Spark session

    # Read your data into a DataFrame (replace "input_file.csv" with your file)
    df = spark.read.csv(f"{root_dir}/flu_by_county.csv", header=True, inferSchema=True)

    # Define the column to group by
    group_column = "date_code"  # Replace with your actual column name

    # Get distinct values in the group column
    distinct_values = [str(row[group_column]) for row in df.select(group_column).distinct().collect()]

    # Group by the column and write each group to a separate file
    for value in distinct_values:
        group_df = df.filter(col(group_column) == value)
        output_path = f"{filecheck2}/{value}.csv"  # Replace with your desired output directory
        group_df.write.csv(output_path, header=True, mode="overwrite")


In [10]:
# Generate Heatmaps
maps_dir = f"{root_dir}/maps"

# Check if the directory is empty
if not os.listdir(maps_dir):
    ca_map = gpd.read_file("CA_Counties/CA_Counties_TIGER2016.shp")
    
    # List all subdirectories in the root directory
    sub_dir = [d for d in os.listdir(filecheck2) if os.path.isdir(os.path.join(filecheck2, d))]

    # Iterate through the subdirectories and read CSV files in each subdirectory
    for sub in sub_dir:
        subdirectory_path = os.path.join(filecheck2, sub)
        csv_files = [f for f in os.listdir(subdirectory_path) if f.endswith('.csv')]

        for csv_file in csv_files:
            csv_file_path = os.path.join(subdirectory_path, csv_file)
            df = pd.read_csv(csv_file_path)
            
            # Calculate the maximum and minimum values in the "Total_ILI" column
            max_value = df["Total_ILI"].max()
            min_value = df["Total_ILI"].min()
            
            #Merge files
            mapf = "COUNTYNS"
            statf = "region"
            map_and_stats = ca_map.merge(df, left_on=mapf, right_on=statf)
            
            #plot map
            fig, ax = plt.subplots(1, figsize=(8, 8))
            plt.xticks(rotation=90)
            map_and_stats.plot(column="Total_ILI", cmap="Reds", linewidth=0.4, ax=ax, edgecolor=".4")
            title_column = "weekending"
            title_value = df[title_column].iloc[0]
            plt.title(f"Total IRI for Week of {title_value}")
            bar_info = plt.cm.ScalarMappable(cmap="Reds", norm=plt.Normalize(vmin=min_value, vmax=max_value))
            bar_info._A = []
            cbar = fig.colorbar(bar_info)
            
            # Save the map as an image file in the "maps" directory
            output_file = os.path.join(maps_dir, os.path.splitext(csv_file)[0] + ".png")
            plt.savefig(output_file, dpi=300)  # Change the dpi and format as needed
            
            # Close the figure to release memory
            plt.close(fig)

In [ ]:
#Predictive map
filecheck3 = f"{root_dir}/flu_by_county.csv"
if os.path.isfile(filecheck3):
    # Read the CSV file into a PySpark DataFrame
    df = spark.read.csv(f"{root_dir}/flu_by_county.csv", header=True, inferSchema=True)
    #df.show()
    # Select the relevant columns
    df = df.select("weekending", "Total_ILI")

    # Convert the PySpark DataFrame to a Pandas DataFrame
    pandas_df = df.toPandas()

    # Convert the "weekending" column to a datetime format
    pandas_df["weekending"] = pd.to_datetime(pandas_df["weekending"])

    # Create the Matplotlib plot
    plt.plot(pandas_df["weekending"], pandas_df["Total_ILI"])

    # Set labels for the x and y axes
    plt.xlabel("Date")
    plt.ylabel("Total_ILI")
    
    df = df.select("weekending", "Total_ILI")
    df = df.withColumnRenamed("Total_ILI", "target")

    # Collect the data to the driver for processing with Pyramid ARIMA
    data = df.select("weekending", "target").orderBy("weekending").toPandas()

    # Train the ARIMA model using the Pyramid ARIMA library
    # You can adjust the ARIMA parameters as needed
    stepwise_model = auto_arima(
        data["target"],
        seasonal=True,
        m=12,  # Define the seasonality (e.g., 12 for monthly data)
        stepwise=True
    )

    # Make predictions
    # You can replace this with your actual future data
    future_data = df.select("weekending").distinct().toPandas()
    predictions, conf_int = stepwise_model.predict(n_periods=future_data.shape[0], return_conf_int=True)

    # Combine predictions with future data
    future_data["predicted_ILI"] = predictions

    # Display or save the predictions
    # You can save the predictions as needed or show them
    future_data.show()


    # Show the plot
    plt.show()

In [ ]:
spark.stop()